In [5]:
import torch
from degradation_model import High2Low
from discriminator import Discriminator
from load_data import test_set_data, Test_Data
from torch.utils.data import DataLoader
import numpy as np
import cv2

In [6]:
test_save = "intermid_results"
ep = 1
w = torch.load("intermid_results/models/model_epoch_001.pth")
G_h2l = High2Low().cuda()
G_h2l.load_state_dict(w['G_h2l'])
D_h2l = Discriminator(16).cuda()
G_h2l.eval()
D_h2l.eval()
D_h2l.load_state_dict(w['D_h2l'])
test_data = test_set_data(Test_Data)
test_loader = DataLoader(dataset=test_data, batch_size=1, shuffle=False)

In [9]:
for i, sample in enumerate(test_loader):
    high_temp = sample["hr"].numpy()
    high = torch.from_numpy(np.ascontiguousarray(high_temp[:, ::-1, :, :])).cuda()
    with torch.no_grad():
        high_gen = G_h2l(high)
    np_gen = high_gen.detach().cpu().numpy().transpose(0, 2, 3, 1).squeeze(0)
    np_gen = (np_gen - np_gen.min()) / (np_gen.max() - np_gen.min())
    np_gen = (np_gen * 255).astype(np.uint8)
    path = "{}/imgs/{}_{}_sr.png".format(test_save, ep, i + 1)
    cv2.imwrite(path, np_gen)